In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name(0)

'GeForce 840M'

In [5]:
torch.cuda.memory_reserved()

0

In [28]:
torch.cuda.memory_allocated()

37376

In [7]:
#using CPU

In [8]:
var1=torch.FloatTensor([1.0,2.0,3.0])

In [9]:
var1

tensor([1., 2., 3.])

In [10]:
var1.device

device(type='cpu')

In [11]:
#now going to use gpu

In [12]:
var1=torch.FloatTensor([1.0,2.0,3.0]).cuda()

In [13]:
var1

tensor([1., 2., 3.], device='cuda:0')

In [14]:
var1.device

device(type='cuda', index=0)

## Creating an ANN using Pytorch

In [15]:
import pandas as pd
df = pd.read_csv('./diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df,hue="Outcome")

In [16]:
X= df.drop("Outcome",axis=1).values   #values makes numpy array
y=df['Outcome'].values 

In [17]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

________________________

## Pytorch

In [18]:
#Libraries from pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
#### Creating Tensors  
X_train = torch.FloatTensor(X_train).cuda() #independet features must be convert to float
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [20]:
X_train.device

device(type='cuda', index=0)

In [ ]:
df.shape

In [22]:
### Creating Model with Pytorch

class ANN_Model(nn.Module):   
    def __init__(self,input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x

In [23]:
#### instantiate my ANN_Model

torch.manual_seed(20)
model = ANN_Model()

In [24]:
model.parameters

<bound method Module.parameters of ANN_Model(
  (f_connected1): Linear(in_features=8, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=2, bias=True)
)>

In [25]:
for i in model.parameters():
    print(i.is_cuda)

False
False
False
False
False
False


In [26]:
model = model.cuda()

In [27]:
for i in model.parameters():
    print(i.is_cuda)

True
True
True
True
True
True


In [ ]:
model.parameters()

In [29]:
#### Backward Propagation - Define the loss function, define optimizer
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [30]:
epochs = 500
final_losses = []

for i in range(epochs):
    i=i+1
    y_pred = model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%10 ==1:
        print("Epoch number is: {} and the loss is: {}".format(i,loss.item()))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

Epoch number is: 1 and the loss is: 3.457210063934326
Epoch number is: 11 and the loss is: 0.8019209504127502
Epoch number is: 21 and the loss is: 0.609032154083252
Epoch number is: 31 and the loss is: 0.5917770862579346
Epoch number is: 41 and the loss is: 0.5679707527160645
Epoch number is: 51 and the loss is: 0.5529042482376099
Epoch number is: 61 and the loss is: 0.5410094261169434
Epoch number is: 71 and the loss is: 0.5310390591621399
Epoch number is: 81 and the loss is: 0.5220361351966858
Epoch number is: 91 and the loss is: 0.5135972499847412
Epoch number is: 101 and the loss is: 0.5061254501342773
Epoch number is: 111 and the loss is: 0.4983409643173218
Epoch number is: 121 and the loss is: 0.4960552752017975
Epoch number is: 131 and the loss is: 0.4828636944293976
Epoch number is: 141 and the loss is: 0.4756036400794983
Epoch number is: 151 and the loss is: 0.47992271184921265
Epoch number is: 161 and the loss is: 0.4793107807636261
Epoch number is: 171 and the loss is: 0.465

In [ ]:
#### plot he loss function
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
final_losses[0]

In [ ]:
fi_los = [fl.item() for fl in final_losses ]
plt.plot(range(epochs),fi_los)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
#### Prediction in X_test data
predictions =[]

with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

In [ ]:
from sklearn.metrics import confusion_matrix
cm =confusion_matrix(y_test,predictions)
cm

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual value')
plt.ylabel('Predicted values')

In [ ]:
from sklearn.metrics import accuracy_score
score =accuracy_score(y_test,predictions)
score

In [ ]:
### Save and Load the model
torch.save(model,'diabetes.pt')

In [ ]:
model = torch.load("./diabetes.pt")

In [ ]:
model.eval()